<a href="https://colab.research.google.com/github/dtabuena/EphysLib/blob/main/InputR_from_IC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
################################## Analyze R-Input In Current Clamp ##################################

def inputR_analysis(abf_recordings_df, protocol_aliases,to_plot=False, verbose = False):
    'Loop through abfs to calculate the input resistance in Current Clamp'
    correct_protocol = [ p in protocol_aliases for p in abf_recordings_df['protocol']]
    for file_name in tqdm( abf_recordings_df.index[correct_protocol] ):
        abf = pyabf.ABF( file_name )
        if len(abf.sweepList)<2:
            # print(list(abf_recordings_df.index).index(file_name),    'not enough sweeps')
            abf_recordings_df.at[file_name,'Rinput_(MO)'] = np.nan
            abf_recordings_df.at[file_name,'Cmf_IC_(pF)'] = np.nan
            continue
            
        inputR_fit = input_res_curve(abf,to_plot=to_plot)
        abf_recordings_df.at[file_name,'Rinput_(MO)'] = inputR_fit['slope']*1000
        if len(abf_recordings_df.at[file_name,'passing_sweeps'])==0:
            abf_recordings_df.at[file_name,'Cmf_IC_(pF)'] = np.nan
            continue
        Cmf_IC = IC_sweep_capacitance_mean(abf,abf_recordings_df.at[file_name,'passing_sweeps'],to_plot=to_plot,verbose=verbose)
        abf_recordings_df.at[file_name,'Cmf_IC_(pF)'] = Cmf_IC
    return abf_recordings_df


def input_res_curve(abf,to_plot=False):
    'Calulates the series of delta Vs and delta Is and fits with a line to find the resistance.'
    from scipy.signal import butter,filtfilt
    stim_currents = []
    ss_voltage = []
    is_base, is_stim = protocol_baseline_and_stim(abf)
    for s in abf.sweepList:
        abf.setSweep(s)
        delta_v, _, _ = sweep_VIR(abf.sweepY, abf.sampleRate, is_stim = is_stim)
        delta_I, _, _    = sweep_VIR(abf.sweepC, abf.sampleRate, is_stim = is_stim) # repurpose but for command current
        stim_currents.append( delta_I)
        ss_voltage.append(delta_v)
    
    inputR_fit = {}
    inputR_fit['slope'], inputR_fit['intercept'] , r_value, p_value, std_err = stats.linregress(stim_currents, ss_voltage)
    inputR_fit['R2'] = r_value**2
    if to_plot:
        plt.scatter(stim_currents, ss_voltage)
        matplotlib.pyplot.text(10, 0, 'Ri = ' + str(round(inputR_fit['slope']*1000)) + 'MO')
    return inputR_fit        

def sweep_VIR(trace,rate,is_stim = None, window_t=0.100):
    'Takes a trace snd calulates the steady state delta V from'
    'a stimulus in Current Clamp'
    if any(is_stim == None):
        is_stim = [True for i in trace]
    base_v = trace[:np.where(is_stim==True)[0][0]]
    cutoff = 5
    nyq = rate/2
    normal_cutoff = cutoff / nyq
    b, a = butter(3, normal_cutoff, btype='low')
    filtered_step_v = filtfilt(b, a, trace[is_stim])
    window_wid = int(window_t*rate)
    med_base_v = np.median(base_v[-window_wid:-1])
    med_stim_v = np.median(filtered_step_v[-window_wid:-1])
    delta_v = med_stim_v - med_base_v
    return delta_v, med_base_v, med_stim_v




def IC_sweep_capacitance_mean(abf,passing_sweeps,to_plot=False, verbose = False):
    'Takes an abf and calulates the membrane capacitance using'
    'methods described in https://journals.physiology.org/doi/epdf/10.1152/jn.00160.2009'
    
    
    is_base, is_stim = protocol_baseline_and_stim(abf)
    rVm_list = []
    V0_list = []
    SS_list = []
    Stim_list = []
    time = abf.sweepX[is_stim]
    time = time - time[0]
    # for s in abf.sweepList:
    for s in passing_sweeps:
        abf.setSweep(s)
        Istim = abf.sweepC[is_stim]
        if not( np.mean(Istim) < 0 ):
            continue
        Vm = abf.sweepY[is_stim]
        SS = np.percentile(Vm,.0001)
        V0 = Vm[0]
        relative_Vm = (Vm - SS) / (V0-SS)
        rVm_list.append(relative_Vm)
        V0_list.append(V0)
        SS_list.append(SS)
        Stim_list.append(np.median(Istim))
    if len(V0_list)<1:
        'likely there are passing sweeps but they are positive current'
        return np.nan
    
    rVm_array = np.stack(rVm_list).T
    V0_array = np.array(V0_list)
    SS_array = np.array(SS_list)
    Stim_array = np.array(Stim_list)
    mean_rVm = np.mean(rVm_array,axis=-1)
    if to_plot:
        plt.plot(time,rVm_array,'gray')
        plt.plot(time,mean_rVm,'k')


    p0 = ( 0.1, 0.8, 0.0001,  0.01, 0)
    bounds=([0,.3,0,.01,-.2], [0.25,1.2,.01,2,.2])

    try:
        fit_params, cv = scipy.optimize.curve_fit(binary_exp, time, mean_rVm, p0, bounds=bounds) #
        amp_1, amp_2, tau_1, tau_2, ss = fit_params
        Vm_hat = binary_exp(time, amp_1, amp_2, tau_1, tau_2, ss)

        Vm_hat1 = binary_exp(time, amp_1, 0, tau_1, tau_2, ss) + amp_2
        Vm_hat2 = binary_exp(time, 0, amp_2,  tau_1, tau_2, ss) + amp_1

        t_slow = tau_2*np.ones_like(V0_array)
        dv_slow = (SS_array - V0_array) * amp_2 *1e-3
        i_stim = Stim_array*1e-12
        r_slow = dv_slow/i_stim
        c_slow = t_slow /r_slow
        Cmf_IC = c_slow*1e12
    except:
        print(V0_array)
        print(Stim_array)
        print(SS_array)
        return np.nan 

    if verbose:
        print('V0', V0_array)
        print('I', i_stim*1e12)
        print('dV', dv_slow*1e3)
        print('SSv', SS_array)
        print('Rm', r_slow*1e-6)
        print('Cm', Cmf_IC)
        print('amp_1, amp_2, tau_1, tau_2, ss')
        print('Fit', fit_params)

    if to_plot:
        plt.plot(time,Vm_hat,'r')
        plt.plot(time,Vm_hat1,'c')
        plt.plot(time,Vm_hat2,'m')
        plt.show()

    return np.mean(Cmf_IC)